In [1]:
#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [2]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [3]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [7]:
dataset['GDP']

1913-01-01   NaN
1913-02-01   NaN
1913-03-01   NaN
1913-04-01   NaN
1913-05-01   NaN
1913-06-01   NaN
1913-07-01   NaN
1913-08-01   NaN
1913-09-01   NaN
1913-10-01   NaN
1913-11-01   NaN
1913-12-01   NaN
1914-01-01   NaN
1914-02-01   NaN
1914-03-01   NaN
1914-04-01   NaN
1914-05-01   NaN
1914-06-01   NaN
1914-07-01   NaN
1914-08-01   NaN
1914-09-01   NaN
1914-10-01   NaN
1914-11-01   NaN
1914-12-01   NaN
1915-01-01   NaN
1915-02-01   NaN
1915-03-01   NaN
1915-04-01   NaN
1915-05-01   NaN
1915-06-01   NaN
              ..
2019-02-22   NaN
2019-02-25   NaN
2019-02-26   NaN
2019-02-27   NaN
2019-02-28   NaN
2019-03-01   NaN
2019-03-04   NaN
2019-03-05   NaN
2019-03-06   NaN
2019-03-07   NaN
2019-03-08   NaN
2019-03-11   NaN
2019-03-12   NaN
2019-03-13   NaN
2019-03-14   NaN
2019-03-15   NaN
2019-03-18   NaN
2019-03-19   NaN
2019-03-20   NaN
2019-03-21   NaN
2019-03-22   NaN
2019-03-25   NaN
2019-03-26   NaN
2019-03-27   NaN
2019-03-28   NaN
2019-03-29   NaN
2019-03-31   NaN
2019-04-01   N

In [8]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [10]:
#converting GDP quarterly data into monthly - need to convert it so it fills in the following 3 months

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [21]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [22]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [23]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-59]

In [24]:
y = dataset['Recession'] #splitting off Y

In [25]:
y_shift = y.shift(-3) #shifting y to forecast 3 months out

In [26]:
y_shift = y_shift.fillna(0)

In [27]:
dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [28]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [37]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [38]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [41]:
X = dataset

In [ ]:
### Data Prep Finished Here ###